## Chassis Example Notebooks
Welcome to the examples section for [Chassis](https://chassis.ml), which contains notebooks that auto-containerize models built using the most common machine learning (ML) frameworks. 

#### What is Chassis?
Chassis allows you to automatically create a Docker container from your model code and push that container image to a Docker registry. All you need is your model loaded into memory and a few lines of Chassis code! Our example bank is here to provide reference examples for many common ML frameworks.  

Can't find the framework you are looking for or need help? Fork this repository and open a PR, or list the desired framework in a new issue. We're always interested in growing this example bank! 

The primary maintainers of Chassis also actively monitor our [Discord Server](https://discord.gg/cHpzY9yCcM), so feel free to join and ask any questions you might have. We'll be there to respond and help out promptly.

In [2]:
import time
import chassisml
import numpy as np
import getpass
import json
from io import StringIO
import pandas as pd
import numpy as np
from sklearn_pmml_model.linear_model import PMMLLinearRegression

## Enter credentials
Dockerhub creds and Modzy API Key

In [3]:
dockerhub_user = getpass.getpass('docker hub username')
dockerhub_pass = getpass.getpass('docker hub password')

docker hub username········
docker hub password········


## Load and Test Model from PMML File

In [4]:
# Prepare data
df = pd.read_csv('data/categorical-test.csv')
Xte = df.iloc[:, 1:]

# Create sample data for testing later
with open("data/sample_regression.csv", "w") as f:
    Xte[:10].to_csv(f, index=False)

In [5]:
# Load model
clf = PMMLLinearRegression(pmml="models/linear-model-lm.pmml")

# Test model
output = clf.predict(Xte)

C:\tools\Anaconda3\envs\modzy\lib\site-packages\sklearn\base.py:413: UserWarning: X has feature names, but OneHotEncoder was fitted without feature names
  warnings.warn(


## Write process function

* Must take bytes as input
* Preprocess bytes, run inference, postprocess model output, return results

In [6]:
def process(input_bytes):
    # load data
    inputs = pd.read_csv(StringIO(str(input_bytes, "utf-8")))
    
    # predictions
    output = clf.predict(inputs)
    
    # process output
    inference_result = {
        "predictions": [
            {"row": i+1, "score": score.item()} for i, score in enumerate(output)
        ]
    }    
    
    
    # format output
    structured_output = {
        "data": {
            "result": inference_result,
            "explanation": None,
            "drift": None,
        }
    }
    return structured_output

## Initialize Chassis Client
We'll use this to interact with the Chassis service

In [7]:
chassis_client = chassisml.ChassisClient("https://chassis.app.modzy.com")

## Create and test Chassis model
* Requires `process_fn` defined above

In [8]:
# create Chassis model
chassis_model = chassis_client.create_model(process_fn=process)

# test Chassis model locally (can pass filepath, bufferedreader, bytes, or text here):
sample_filepath = './data/sample_regression.csv'
results = chassis_model.test(sample_filepath)
print(results)

b'{"data":{"result":{"predictions":[{"row":1,"score":0.9590476609154883},{"row":2,"score":0.3066354811397165},{"row":3,"score":0.6510299850602321},{"row":4,"score":0.9548748798188142},{"row":5,"score":0.8892685958181936},{"row":6,"score":0.8744135392564563},{"row":7,"score":0.5529119650921299},{"row":8,"score":0.7939719287317026},{"row":9,"score":0.5676047268515574},{"row":10,"score":0.6944419835236506}]},"explanation":null,"drift":null}}'


C:\tools\Anaconda3\envs\modzy\lib\site-packages\sklearn\base.py:413: UserWarning: X has feature names, but OneHotEncoder was fitted without feature names
  warnings.warn(


In [9]:
# manually construct conda environment to pass to Chassis job

# NOTE: if you define your environment manually like this, the "chassisml" package must be included within the pip depedencies
env = {
    "name": "sklearn-chassis",
    "channels": ['conda-forge'],
    "dependencies": [
        "python=3.8.5",
        {
            "pip": [
                "sklearn-pmml-model",
                "numpy",
                "chassisml",
                "pandas"
            ] 
        }
    ]
}

## Publish model to Docker
Need to provide model name, model version, and Dockerhub credentials

In [10]:
MODEL_NAME = "PMML Sample Linear Regression"
start_time = time.time()
response = chassis_model.publish(
    model_name=MODEL_NAME,
    model_version="0.0.1",
    registry_user=dockerhub_user,
    registry_pass=dockerhub_pass,
    conda_env=env
)

Starting build job... Ok!


In [11]:
job_id = response.get('job_id')
final_status = chassis_client.block_until_complete(job_id)
end_time = time.time()
if final_status['status']['succeeded'] == 1:
    print("Job Completed in {} minutes.\n\nView your new container image here: https://hub.docker.com/repository/docker/{}/{}".format(round((end_time-start_time)/60, 5), dockerhub_user, "-".join(MODEL_NAME.lower().split(" "))))
else:
    print("Job Failed. See logs below:\n\n{}".format(final_status['logs']))

Job Completed in 4.85596 minutes.

View your new container image here: https://hub.docker.com/repository/docker/bmunday131/pmml-sample-linear-regression
